In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [ ]:
model_name = 'faceDetectModel'
savedModelPath = '../log/saved_models/faceDetectModel'
tb_log_dir = "../log/tensorboard/faceDetectModel"
cp_filepath = '../log/cps/faceDetectModel/latest_weights.h5'

batch_size = 32
epochs = 10

## Model callbacks

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
                    # Checkpoint callback                    
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

                    # Tensorboard callback
                    tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1)
            ]

## Data parameters

In [ ]:
image_height = 224
image_width = 224

## Creating the training dataset

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

## Creating the validation dataset

In [ ]:
validation_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

## Creating the test dataset

## Load the ResNet model

In [ ]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
resnet = ResNet50(
                include_top=False,
                weights=None,  # No pretrained weights
                input_shape=(image_height, image_width, 3)
                )
resnet.summary()

## Adding Pooling and Fully Connected Layer for binary classification

In [ ]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Running base model in inference mode
x = resnet(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.Flatten()(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

# Loading latest saved weights into model
# if(os.path.isfile(cp_filepath)):
#     print('Loading weights...')
#     model.load_weights(filepath=cp_filepath)

## Training the model with the dataset

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
            face_training_ds, 
            epochs=epochs, 
            callbacks=callbacks, 
            validation_data=validation_ds
        )

## Save the model

In [ ]:
model.save(savedModelPath)

## Test model with test dataset

## Predict new images

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/FaceTest/000001.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)
score = preds[0][0]

if(score > 0.5):
    print("This image contains no face. ({:.2f} percent confidence)".format(100 * score))
else:
    print("This image contains a face. ({:.2f} percent confidence)".format(100 * (1-score)))